In [13]:
from trainer.data import load_data
from trainer.policy import get_model
from trainer.env import make_env
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from stable_baselines3 import PPO
from trade_tester.tester import render_candles

### Load data

In [2]:
load_data_kwargs = dict(
    path = 'klines/DOGEUSDT_15m.csv',
    preprocessing_kwargs = dict(
        bb = dict(price='close', period=20, deviation=2.0),
    ),
    split_validate_percent = 0,
    last_n = 1e4,
)
train_klines, val_klines, indicators = load_data(**load_data_kwargs)

In [3]:
train_klines

,date,open,high,low,close,vol,trades,bb_upper,bb_middle,bb_lower
81153,2022-11-02 17:15:00,0.13219,0.13367,0.13149,0.13311,295942803,49308,0.133092,0.130541,0.127990
81154,2022-11-02 17:30:00,0.13310,0.13310,0.13192,0.13240,266350632,43247,0.133255,0.130725,0.128195
81155,2022-11-02 17:45:00,0.13239,0.13253,0.12900,0.13126,605500208,100544,0.133196,0.130860,0.128523
81156,2022-11-02 18:00:00,0.13125,0.13880,0.12889,0.13637,2150869185,361760,0.134489,0.131183,0.127877
81157,2022-11-02 18:15:00,0.13638,0.13843,0.13600,0.13698,734171861,132961,0.135634,0.131544,0.127453
...,...,...,...,...,...,...,...,...,...,...
91148,2023-02-14 20:00:00,0.08343,0.08355,0.08320,0.08326,45774631,4874,0.083679,0.082924,0.082168
91149,2023-02-14 20:15:00,0.08326,0.08341,0.08318,0.08330,35669079,3674,0.083715,0.082942,0.082168
91150,2023-02-14 20:30:00,0.08330,0.08331,0.08318,0.08325,21990887,2663,0.083736,0.082979,0.082222
91151,2023-02-14 20:45:00,0.08325,0.08337,0.08319,0.08329,46051471,4138,0.083767,0.083004,0.082241


In [4]:
env_kwargs = dict(
    env_class='TradingEnv2Actions',
    tester='BBFutureTester2',
    klines=train_klines,
    indicators=indicators,
)
env = make_env(**env_kwargs)
# model = get_model(env=env)
# model = PPO('MlpPolicy', env)

In [11]:
env.envs[0].tester.klines

,date,open,high,low,close,vol,trades,bb_upper,bb_middle,bb_lower
86990,2023-01-02 12:30:00,0.07197,0.07203,0.07189,0.07194,39644120,3038,0.072276,0.071937,0.071598
86991,2023-01-02 12:45:00,0.07194,0.07215,0.07191,0.07193,77712736,4536,0.072276,0.071945,0.071613
86992,2023-01-02 13:00:00,0.07194,0.07203,0.07186,0.07196,31171126,2931,0.072228,0.071930,0.071631
86993,2023-01-02 13:15:00,0.07196,0.07214,0.07188,0.07213,42789164,3162,0.072231,0.071931,0.071630
86994,2023-01-02 13:30:00,0.07213,0.07214,0.07152,0.07162,146923647,8612,0.072220,0.071905,0.071590
...,...,...,...,...,...,...,...,...,...,...
88285,2023-01-16 00:15:00,0.08661,0.08692,0.08524,0.08541,206010381,18945,0.086839,0.085822,0.084805
88286,2023-01-16 00:30:00,0.08542,0.08635,0.08538,0.08589,66379368,7573,0.086844,0.085847,0.084850
88287,2023-01-16 00:45:00,0.08589,0.08714,0.08588,0.08662,112029980,11884,0.086933,0.085912,0.084891
88288,2023-01-16 01:00:00,0.08662,0.08754,0.08648,0.08731,190098937,18232,0.087146,0.086021,0.084895


In [14]:
render_candles(env.envs[0].tester.klines)

In [ ]:
model.policy._modules

In [ ]:
# Извлеките параметры политики
policy_parameters = model.policy.state_dict()
# Выведите веса каждого слоя
for name, weights in policy_parameters.items():
    print(f"{name}: {weights}")

In [ ]:
# Исследование структуры model.policy
def explore_structure(obj, indent=0):
    for name in dir(obj):
        if not name.startswith("__"):
            value = getattr(obj, name)
            if not callable(value):
                print("  " * indent, f"{name}: {type(value)}")
                if isinstance(value, torch.nn.Module):
                    explore_structure(value, indent + 1)

# print("Policy structure:")
# explore_structure(model.policy)

In [ ]:
obs = env.reset()
obs = th.tensor(obs)
obs = obs.transpose(1, 2)[0]
for channel in range(obs.shape[0]):
    print()

In [ ]:
from trainer.nn import CustomNN

nn = CustomNN(np.array([[9, 300], [9, 300]]))

In [ ]:
nn.named_parameters

In [ ]:
for name, param in nn.named_parameters():
    num_params = param.numel()
    print(num_params)

In [3]:
import pandas as pd
a = pd.DataFrame([[1, 2, 3], [2, 3, 4], [5, 6, 7]])
a

,0,1,2
0,1,2,3
1,2,3,4
2,5,6,7
